In [2]:
import requests
import json
import pandas as pd
from transformers import pipeline
from PIL import Image, ImageDraw
from io import BytesIO

ImportError: cannot import name 'pipeline' from 'transformers' (c:\Users\vanne\anaconda3\Lib\site-packages\transformers\__init__.py)

In [ ]:
data = requests.get('https://api.data.gov.sg/v1/transport/traffic-images')
results = json.loads(data.text)

In [ ]:
cameras = results['items'][0]['cameras']

# Flatten the nested dictionary structure
camera_data = []
for camera in cameras:
    camera_info = {
        'timestamp': camera['timestamp'],
        'image': camera['image'],
        'latitude': camera['location']['latitude'],
        'longitude': camera['location']['longitude'],
        'camera_id': camera['camera_id'],
        'image_height': camera['image_metadata']['height'],
        'image_width': camera['image_metadata']['width'],
        'image_md5': camera['image_metadata']['md5']
    }
    camera_data.append(camera_info)

# Create a DataFrame from the flattened data
df = pd.DataFrame(camera_data)
print(df)

In [ ]:
## Hugging face api
images = df['image'].tolist()
response = requests.get(images[1])

image = Image.open(BytesIO(response.content)).convert('RGB')

checkpoint = "google/owlv2-base-patch16-ensemble"
detector = pipeline(model=checkpoint, task="zero-shot-object-detection")

In [ ]:
prediction = detector(image, candidate_labels=['cars'])
print('prediction', prediction)

In [ ]:
def count_verified_cars(image):
    verified_cars = 0
    for car in image:
        if car['score'] > 0.12:
            verified_cars += 1
    return verified_cars

verified_cars = count_verified_cars(prediction)
print(verified_cars)

In [ ]:
draw = ImageDraw.Draw(image)

for p in prediction:
    box = p['box']  
    label = p['label']
    score = p['score']

    xmin, ymin, xmax, ymax = box.values()
    draw.rectangle((xmin, ymin, xmax, ymax), outline='red', width=1)
    draw.text((xmin, ymin), f'{label}: {round(score, 2)}', fill='white')

image_filename = 'annotated_image.jpg'
image.save(image_filename)


In [ ]:
# from flask import Flask, request, jsonify
# import requests

# app = Flask(__name__)

# @app.route('/congestion', methods=['POST'])
# def get_congestion_data():
#     data = request.json
#     lon, lat = data['longitude'], data['latitude']
    
#     # Call data.gov.sg API
#     traffic_data = requests.get('https://api.data.gov.sg/v1/transport/traffic-images').json()
#     image_url = get_image_url(traffic_data, lon, lat)  # Implement this function to get relevant image URL

#     # Call AI Model for car count
#     car_count = call_ai_model(image_url)  # Implement this function to use OpenAI/HuggingFace API
    
#     # Determine congestion level
#     congestion_level = categorize_congestion(car_count)  # Implement this function to categorize
    
#     # Call Location API for address
#     location = get_location(lon, lat)  # Implement this function to call reverse geocoding API
    
#     return jsonify({
#         'car_count': car_count,
#         'congestion_level': congestion_level,
#         'location': location
#     })

# def get_image_url(traffic_data, lon, lat):
#     # Logic to find the closest traffic camera image based on lon/lat
#     pass

# def call_ai_model(image_url):
#     # Logic to send image to AI model and get car count
#     pass

# def categorize_congestion(car_count):
#     if car_count < 10:
#         return 1
#     elif car_count < 20:
#         return 2
#     elif car_count < 30:
#         return 3
#     else:
#         return 4

# def get_location(lon, lat):
#     response = requests.get(f'https://api.location.service/reverse-geocode?lon={lon}&lat={lat}')
#     return response.json()['location']

# if __name__ == '__main__':
#     app.run(debug=True)


In [6]:
data = requests.get('https://api.data.gov.sg/v1/transport/traffic-images')
results = json.loads(data.text)

In [8]:
## Hugging face api
images = df['image'].tolist()
response = requests.get(images[1])

image = Image.open(BytesIO(response.content)).convert('RGB')

checkpoint = "google/owlv2-base-patch16-ensemble"
detector = pipeline(model=checkpoint, task="zero-shot-object-detection")

In [1]:
draw = ImageDraw.Draw(image)

for p in prediction:
    box = p['box']  
    label = p['label']
    score = p['score']

    xmin, ymin, xmax, ymax = box.values()
    draw.rectangle((xmin, ymin, xmax, ymax), outline='red', width=1)
    draw.text((xmin, ymin), f'{label}: {round(score, 2)}', fill='white')

image_filename = 'annotated_image.jpg'
image.save(image_filename)


NameError: name 'ImageDraw' is not defined